# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [10]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal

#### Creating list of filepaths to process original event csv data files

In [19]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/foz/Code/udacity/data_engineering_nanodegree/01B_data_modelling_with_apache_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [20]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
        
            full_data_rows_list.append(line) 
        
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [21]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

401


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Setup cluster and keyspace

#### Creating a Cluster

In [22]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [23]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [24]:
session.set_keyspace('sparkify')

### Querying songs by session and item in session

This ETL creates a table to find songs by session and item in session. It will return the artist name, song name, and the song length.

In [25]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_items_in_session_table = """
CREATE TABLE IF NOT EXISTS session_items
(session_id int, item_in_session int, artist text, song_title text, song_length decimal, PRIMARY KEY(session_id, item_in_session))
"""

session.execute(create_items_in_session_table)



In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    for line in csvreader:
        query = """
        INSERT INTO session_items (session_id, item_in_session, artist, song_title, song_length) VALUES (%s, %s, %s, %s, %s)
        """
        session.execute(
            query, ( 
                int(line['sessionId']), 
                int(line['itemInSession']),
                line['artist'], 
                line['song'], 
                Decimal(line['length'])
            )
        )

In [28]:
info_by_session_ids_query = """
SELECT artist, song_title, song_length FROM session_items WHERE session_id = 338 AND item_in_session = 4
"""

rows = session.execute(info_by_session_ids_query)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length,)

### Querying songs by user session

This ETL creates a table to view the songs by user sessions. It's ordered by `item_in_session` then by user's first and last name.

In [30]:
create_songs_by_user_query = """
CREATE TABLE IF NOT EXISTS songs_by_user_session
(user_id int, session_id int, item_in_session int, user_first_name text, user_last_name text, artist text, song_title text,
PRIMARY KEY((user_id, session_id), item_in_session, user_first_name, user_last_name))
"""

session.execute(create_songs_by_user_query)


In [31]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    for line in csvreader:
        query = """
        INSERT INTO songs_by_user_session (user_id, session_id, item_in_session, user_first_name, user_last_name, artist, song_title) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        session.execute(
            query, (
                int(line['userId']),
                int(line['sessionId']),
                int(line['itemInSession']), 
                line['firstName'],
                line['lastName'],
                line['artist'], 
                line['song']
            )
        )


In [32]:
songs_by_user_query = """
SELECT item_in_session, user_first_name, user_last_name, artist, song_title 
FROM songs_by_user_session 
WHERE user_id = 10 AND session_id = 182
"""

rows = session.execute(songs_by_user_query)
    
for row in rows:
    print (row.artist, row.song_title, row.item_in_session, row.user_first_name, row.user_last_name,)


### Querying users by song

This ETL creates a table to view the all the users who listened to a given song.

In [42]:
create_users_by_song_query = """
CREATE TABLE IF NOT EXISTS users_by_song
(song text, user_id int, user_first_name text, user_last_name text, 
PRIMARY KEY(song, user_id))
"""

session.execute(create_users_by_song_query)                   

In [43]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    for line in csvreader:
        query = """
        INSERT INTO users_by_song (song, user_id, user_first_name, user_last_name) 
        VALUES (%s, %s, %s, %s)
        """
        session.execute(
            query, (
                line['song'],
                int(line['userId']),
                line['firstName'],
                line['lastName']
            )
        )


In [44]:
users_by_song_query = """
SELECT user_id, user_first_name, user_last_name 
FROM users_by_song
WHERE song = 'All Hands Against His Own'
"""

rows = session.execute(users_by_song_query)
    
for row in rows:
    print (row.user_first_name, row.user_last_name,)

Jayden Graves


### Drop the tables before closing out the sessions

In [41]:
drop_table = """
DROP TABLE users_by_song;
"""

session.execute(drop_table)
drop_table = """
DROP TABLE songs_by_user_session;
"""

session.execute(drop_table)
drop_table = """
DROP TABLE session_items;
"""

session.execute(drop_table)

### Close the session and cluster connection¶

In [49]:
session.shutdown()
cluster.shutdown()